In [1]:
using Oceananigans
using Oceananigans.Models: ShallowWaterModel
using NCDatasets, Printf, CairoMakie

In [2]:
using FFTW
function mode(y,n)
    N = length(y)
    ŷ = fft(y)
    ŷ[(1:N .≠ n+1) .& (1:N .≠ N+1-n)] .= 0.0 + 0.0im
    return real.(ifft(ŷ))
end

mode (generic function with 1 method)

### Propagation of shallow water wave from initial disturbance

In [3]:
Lx = 20e5
grid = RectilinearGrid(size = (512),
                       x = (-Lx/2, Lx/2),
                       topology = (Periodic, Flat, Flat))

gravitational_acceleration = 9.81
coriolis = FPlane(f=0)

model = ShallowWaterModel(; grid, coriolis, gravitational_acceleration,
                          timestepper = :RungeKutta3,
                          momentum_advection = WENO())

η₀ = 1. #m
x₀ = 0  #m
σ = 50e4
H = 4e3
hᵢ(x) = H + η₀*(x>x₀-σ/2)*(x<x₀+σ/2)*exp(-1/(1-(x/(σ/2))^2))/0.37
set!(model, h = hᵢ)


simulation = Simulation(model, Δt = 1, stop_time = 6*3600);

uh, vh, h = model.solution

fields_filename = joinpath(@__DIR__, "../data/shallow_water_1d.nc")
simulation.output_writers[:fields] = NetCDFWriter(model, (; uh, vh, h),
                                                        filename = fields_filename,
                                                        schedule = TimeInterval(60),
                                                        overwrite_existing = true)


run!(simulation)

┌ Warning: The ShallowWaterModel is currently unvalidated, subject to change, and should not be used for scientific research without adequate validation.
└ @ Oceananigans.Models.ShallowWaterModels ~/.julia/packages/Oceananigans/Rb6LJ/src/Models/ShallowWaterModels/shallow_water_model.jl:129
┌ Warning: Overwriting existing /Users/henrifdrake/code/ESS130/notebooks/shallow_water_1d.nc.
└ @ OceananigansNCDatasetsExt ~/.julia/packages/Oceananigans/Rb6LJ/ext/OceananigansNCDatasetsExt.jl:1023
[ Info: Initializing simulation...
[ Info:     ... simulation initialization complete (2.186 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (1.909 seconds).
[ Info: Simulation is stopping after running for 0 seconds.
[ Info: Simulation time 6 hours equals or exceeds stop time 6 hours.


In [4]:
fig = Figure(size = (850, 300))

axis_kwargs = (xlabel = L"$x$", ylabel = L"$\eta$", limits=((-Lx/2, Lx/2),(-0.1, η₀+0.1)))
ax_h  = Axis(fig[2, 1]; axis_kwargs...)

n = Observable(1)

ds = NCDataset(simulation.output_writers[:fields].filepath, "r")

x = ds["x_caa"];
times = ds["time"][:]

η = @lift ds["h"][:, $n] .- H
hm_h = lines!(ax_h, x, η, linewidth=5, color=:black)

title = @lift @sprintf("t = %.1f", times[$n])
fig[1, 1] = Label(fig, title, fontsize=16, tellwidth=false)
resize_to_layout!(fig)

frames = 1:length(times)

record(fig, "../movies/Lecture14_shallow_water_waves_1d.mp4", frames, framerate=12) do i
    n[] = i
end

η = ds["h"][:,1] .- H
N = length(η)
for n in append!(collect(0:10), [N÷2-1])

    fig = Figure(size = (850, 300))
    axis_kwargs = (xlabel = L"$x$", ylabel = L"$η(x,t)$", limits=((-Lx/2, Lx/2),(-0.1, η₀+0.1)))
    ax  = Axis(fig[1, 1]; axis_kwargs...)
    
    lines!(ax, ds["x_caa"], η, linewidth=10, color=:black, alpha=0.5)

    if n==0
        save("../figures/Lecture14_SW_full.png", fig)
    end
    
    η_accum = zeros(size(η))
    for i in 0:n-1
        η_accum .+= mode(η,i)
        lines!(ax, ds["x_caa"], η_accum, linewidth=2, alpha=0.3)
    end
    η_accum .+= mode(η,n)
    lines!(ax, ds["x_caa"], η_accum, linewidth=2, alpha=1.)
    resize_to_layout!(fig)
    save(string("../figures/Lecture14_SW_",n,".png"), fig)
end
fig

close(ds)

closed Dataset

### Weird propogating shallow water wave

In [5]:
model = ShallowWaterModel(; grid, coriolis, gravitational_acceleration,
                          timestepper = :RungeKutta3,
                          momentum_advection = WENO())

η₀ = 1
x₀ = 0
σ = 10e4
x₁ = 6e4
η₁ = 0.5
η₂ = 0.25
σ₂ = 25e4
x₂ = -15e4
hᵢ(x) = H + (
    η₀*(x>x₀-σ/2)*(x<x₀+σ/2)*exp(-1/(1-(x/(σ/2))^2))/0.37 +
    η₁*(x>x₁-σ/2)*(x<x₁+σ/2)*exp(-1/(1-((x-x₁)/(σ/2))^2))/0.37 +
    η₂*(x>x₂-σ₂/2)*(x<x₂+σ₂/2)*exp(-1/(1-((x-x₂)/(σ₂/2))^2))/0.37
)
set!(model, h = hᵢ)

simulation = Simulation(model, Δt = 0.25, stop_time = 6*3600);

uh, vh, h = model.solution

fields_filename = joinpath(@__DIR__, "../data/shallow_water_weird_1d.nc")
simulation.output_writers[:fields] = NetCDFWriter(model, (; uh, vh, h),
                                                        filename = fields_filename,
                                                        schedule = TimeInterval(60),
                                                        overwrite_existing = true)

run!(simulation)

┌ Warning: The ShallowWaterModel is currently unvalidated, subject to change, and should not be used for scientific research without adequate validation.
└ @ Oceananigans.Models.ShallowWaterModels ~/.julia/packages/Oceananigans/Rb6LJ/src/Models/ShallowWaterModels/shallow_water_model.jl:129
┌ Warning: Overwriting existing /Users/henrifdrake/code/ESS130/notebooks/shallow_water_weird_1d.nc.
└ @ OceananigansNCDatasetsExt ~/.julia/packages/Oceananigans/Rb6LJ/ext/OceananigansNCDatasetsExt.jl:1023
[ Info: Initializing simulation...
[ Info:     ... simulation initialization complete (944.635 ms)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (809.040 ms).
[ Info: Simulation is stopping after running for 0 seconds.
[ Info: Simulation time 6 hours equals or exceeds stop time 6 hours.


In [7]:
fig = Figure(size = (800, 300))

axis_kwargs = (xlabel = L"$x$", ylabel = L"$\eta$", limits=((-Lx/2, Lx/2),(-0.1, η₀+0.1)))
ax_h  = Axis(fig[2, 1]; axis_kwargs...)

n = Observable(1)

ds = NCDataset(simulation.output_writers[:fields].filepath, "r")

x = ds["x_caa"];
times = ds["time"][:]

η = @lift ds["h"][:, $n] .- H
hm_h = lines!(ax_h, x, η)

title = @lift @sprintf("t = %.1f", times[$n])
fig[1, 1] = Label(fig, title, fontsize=16, tellwidth=false)

frames = 1:length(times)

record(fig, "../movies/Lecture14_shallow_water_waves_weird_1d.mp4", frames, framerate=12) do i
    n[] = i
end

close(ds)

closed Dataset

### Rotating

In [9]:
2π/(86400)*cos(2π/360 * 40)

5.570832395430244e-5

In [11]:
Lx = 2e6
grid = RectilinearGrid(size = (256, 256),
                       x = (-Lx/2, Lx/2),
                       y = (-Lx/2, Lx/2),
                       topology = (Periodic, Periodic, Flat))

gravitational_acceleration = 9.81
coriolis = FPlane(f=2π/(86400)*cos(2π/360 * 40))

model = ShallowWaterModel(; grid, coriolis, gravitational_acceleration,
                          timestepper = :RungeKutta3,
                          momentum_advection = WENO())

η₀ = 1. #m
x₀,y₀ = 0,0  #m
σ = 5e5
H = 4e3
hᵢ(x,y) = H + η₀*(
    (x>x₀-σ/2)*(x<x₀+σ/2)*exp(-1/(1-(x/(σ/2))^2)) *
    (y>y₀-σ/2)*(y<y₀+σ/2)*exp(-1/(1-(y/(σ/2))^2)) /
    (0.37^2)
)
set!(model, h = hᵢ)

simulation = Simulation(model, Δt = 1., stop_time = 6*3600);

uh, vh, h = model.solution

fields_filename = joinpath(@__DIR__, "../data/shallow_water_2d_midlat.nc")
simulation.output_writers[:fields] = NetCDFWriter(model, (; uh, vh, h),
                                                        filename = fields_filename,
                                                        schedule = TimeInterval(60),
                                                        overwrite_existing = true)


run!(simulation)

┌ Warning: The ShallowWaterModel is currently unvalidated, subject to change, and should not be used for scientific research without adequate validation.
└ @ Oceananigans.Models.ShallowWaterModels ~/.julia/packages/Oceananigans/Rb6LJ/src/Models/ShallowWaterModels/shallow_water_model.jl:129
[ Info: Initializing simulation...
[ Info:     ... simulation initialization complete (1.856 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (2.196 seconds).
[ Info: Simulation is stopping after running for 0 seconds.
[ Info: Simulation time 6 hours equals or exceeds stop time 6 hours.


In [16]:
fig = Figure(size = (600, 600))

axis_kwargs = (xlabel = "x", ylabel = "y", limits=((-Lx/2, Lx/2), (-Lx/2, Lx/2)))
ax  = Axis(fig[2, 1]; title = L"elevation $\eta$", axis_kwargs...)

n = Observable(1)

ds = NCDataset(simulation.output_writers[:fields].filepath, "r")

x, y = ds["x_caa"], ds["y_aca"];
times = ds["time"][:]

η = @lift ds["h"][:, :, $n] .- H
hm_h = heatmap!(ax_h, x, y, η, colorrange = (0, η₀), colormap = :tempo)
Colorbar(fig[2, 2], hm_h)

title = @lift @sprintf("t = %.1f", times[$n])
fig[1, 1:2] = Label(fig, title, fontsize=14, tellwidth=false)
fig

close(ds)

closed Dataset

In [ ]:
ds = NCDataset(simulation.output_writers[:fields].filepath, "r")
ds["h"]

In [ ]:
heatmap(

In [ ]:
fig = Figure(size = (700, 600))

axis_kwargs = (xlabel = "x", ylabel = "y",)
ax  = Axis(fig[1, 1]; title = L"elevation $\eta$", axis_kwargs...)
hm = heatmap!(ax, ds["h"][:,:,1,240], colorrange=(4000-0.2, 4000.2), colormap=:balance)
Colorbar(fig[1,2], hm)
fig

In [ ]:
ds = NCDataset(simulation.output_writers[:fields].filepath, "r")
heatmap(ds["uh"][:,1,1,:])

In [ ]:
grid = RectilinearGrid(size = (512, 512),
                       x = (-π, π),
                       y = (-π, π),
                       topology = (Bounded, Bounded, Flat))

In [ ]:
gravitational_acceleration = 9.81

In [ ]:
coriolis = FPlane(f=0)

In [ ]:
model = ShallowWaterModel(; grid, coriolis, gravitational_acceleration,
                          timestepper = :RungeKutta3,
                          momentum_advection = WENO())

In [ ]:
bump(x,y,h₀=0.1,x₀=0,y₀=0,σ=0.1) = h₀*exp(-(((x-x₀)/(2σ))^2 + ((y-y₀)/(2σ))^2)/2)
hᵢ(x,y) = 1 + bump(x,y,0.1,-π/20,2) + bump(x,y,0.05,sqrt(5),π/4,0.05) + bump(x,y,0.75,1,0,0.05) + bump(x,y,0.1,-sqrt(2.),-0.5,0.15)
set!(model, h = hᵢ)

In [ ]:
simulation = Simulation(model, Δt = 1e-4, stop_time = 3);

In [ ]:
uh, vh, h = model.solution

fields_filename = joinpath(@__DIR__, "../data/shallow_water_test.nc")
simulation.output_writers[:fields] = NetCDFOutputWriter(model, (; uh, vh, h),
                                                        filename = fields_filename,
                                                        schedule = TimeInterval(0.01),
                                                        overwrite_existing = true)

In [ ]:
run!(simulation)

In [ ]:
fig = Figure(size = (600, 600))

axis_kwargs = (xlabel = "x", ylabel = "y")
ax_h  = Axis(fig[2, 1]; title = "depth h", axis_kwargs...)

n = Observable(1)

ds = NCDataset(simulation.output_writers[:fields].filepath, "r")

x, y = ds["x_caa"], ds["y_aca"];
times = ds["time"][:]

h = @lift ds["h"][:, :, 1, $n]
hm_h = heatmap!(ax_h, x, y, h, colorrange = (1, 1.05), colormap = :tempo)
Colorbar(fig[2, 2], hm_h)

title = @lift @sprintf("t = %.1f", times[$n])
fig[1, 1:2] = Label(fig, title, fontsize=20, tellwidth=false)
fig

In [ ]:
frames = 1:length(times)

record(fig, "shallow_water_waves.mp4", frames, framerate=12) do i
    n[] = i
end